# 5. Моделирование и эксперименты

## 🎯 Цели этапа
1. Обучить и сравнить различные ML-алгоритмы
2. Провести подбор гиперпараметров для лучших моделей
3. Выбрать оптимальную модель для прогнозирования оттока
4. Проанализировать feature importance

## 📊 Данные для моделирования
- Train: 12,802 наблюдений, 23 признака (после SMOTE)
- Test: 2,001 наблюдение, 23 признака (оригинальное распределение)
- Дисбаланс: 50/50 в train, 80/20 в test

In [1]:
import joblib

import sys
import os

os.chdir('..')

sys.path.insert(0, 'src')

from model_training import TrainModels

os.chdir('notebooks')

X_train = joblib.load('../data/processed/X_train.pkl')
X_test = joblib.load('../data/processed/X_test.pkl')
y_train = joblib.load('../data/processed/y_train.pkl')
y_test = joblib.load('../data/processed/y_test.pkl')

trainer = TrainModels(X_train, X_test, y_train, y_test)

baseline_models = trainer.fit_baseline_models()

predictions_baseline_models = trainer.evaluate()

for model_name, metric in predictions_baseline_models.items():
    print(f'Модель: {model_name}. Результаты:')
    print(f'ROC-AUC score: {metric["roc_auc_score"]}')
    print(f'F1-score: {metric["f1_score"]}')
    print(f'\nClassification report:\n{metric["classification_report"]}')
    print('-'*50)

Модель: LogisticRegression. Результаты:
ROC-AUC score: 0.7929652400822225
F1-score: 0.5303454715219421

Classification report:
              precision    recall  f1-score   support

           0       0.91      0.76      0.83      1593
           1       0.43      0.70      0.53       408

    accuracy                           0.75      2001
   macro avg       0.67      0.73      0.68      2001
weighted avg       0.81      0.75      0.77      2001

--------------------------------------------------
Модель: RandomForestClassifier. Результаты:
ROC-AUC score: 0.8480045973191537
F1-score: 0.5886792452830188

Classification report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.90      1593
           1       0.60      0.57      0.59       408

    accuracy                           0.84      2001
   macro avg       0.75      0.74      0.74      2001
weighted avg       0.83      0.84      0.83      2001

-------------------------------------